# Setup

In [13]:
from dotenv import load_dotenv
import os
from pathlib import Path
import requests


In [10]:
load_dotenv(Path(os.getcwd()).parent / ".env")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [15]:
import requests

# Configuración de la API
BASE_URL = 'https://apis-sandbox.bancosantander.es/canales-digitales/sb'
#REDIRECT_URI = '{{REDIRECT_URI}}'  # Reemplaza con tu URI de redireccionamiento
RESPONSE_TYPE = 'CODE'
#STATE = '{{APPLICATION_STATE}}'  # Puedes definir un estado de aplicación personalizado

# Información de autenticación
USERNAME = '00000001A'  # Reemplaza con el nombre de usuario del cliente
PASSWORD = '12345678'  # Reemplaza con la contraseña del cliente
OTP = '000000'  # Reemplaza con el código OTP del cliente

# Obtiene la URL de inicio del flujo Santander Connect
def get_santander_connect_url():
    url = f'{BASE_URL}/prestep-authorize?client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}&response_type={RESPONSE_TYPE}&state={STATE}'
    response = requests.get(url)
    return response.url

# Obtiene los tokens de acceso y actualización
def get_tokens(code):
    url = f'{BASE_URL}/v2/token'
    data = {
        'code': code,
        'grant_type': 'authorization_code',
        'client_id': CLIENT_ID,
        'redirect_uri': REDIRECT_URI
    }
    response = requests.post(url, data=data)
    return response.json()

# Obtiene el access token y el refresh token utilizando el código de autorización
def authenticate():
    santander_connect_url = get_santander_connect_url()
    print(f'Por favor, abre la siguiente URL en tu navegador y autentícate:\n{santander_connect_url}')
    code = input('Ingresa el código de autorización: ')
    tokens = get_tokens(code)
    access_token = tokens['access_token']
    refresh_token = tokens['refresh_token']
    return access_token, refresh_token

# Solicita la autorización para acceder a los scopes necesarios
def authorize(access_token):
    url = f'{BASE_URL}/v2/authorize?client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}&response_type={RESPONSE_TYPE}&state={STATE}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.post(url, headers=headers)
    return response.json()

# Obtiene un nuevo access token utilizando el refresh token
def get_new_access_token(refresh_token):
    url = f'{BASE_URL}/v2/token'
    data = {
        'refresh_token': refresh_token,
        'grant_type': 'refresh_token',
        'client_id': CLIENT_ID
    }
    response = requests.post(url, data=data)
    return response.json()['access_token']

# Obtiene la información de transacciones del cliente
def get_transactions(access_token):
    url = f'{BASE_URL}/v2/card-accounts'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response.json()

# Flujo principal
def main():
    # Autenticación y autorización
    access_token, refresh_token = authenticate()
    authorization_response = authorize(access_token)

    if authorization_response.get('code') == 'AUTHORIZED':
        # Llamada a servicio de Cards
        transactions = get_transactions(access_token)
        print(transactions)
    else:
        print('No se pudo obtener la autorización para acceder a los scopes necesarios.')


In [ ]:
main()

Por favor, abre la siguiente URL en tu navegador y autentícate:
https://apis-sandbox.bancosantander.es/canales-digitales/sb/prestep-authorize?client_id=%7B%7BCLIENT_ID%7D%7D&redirect_uri=%7B%7BREDIRECT_URI%7D%7D&response_type=CODE&state=%7B%7BAPPLICATION_STATE%7D%7D
